# Biases, imbalance and pre-processing

Today we will be looking at how to address biases arising from our dataset due to class imbalance, as well as how to get the most out of the data we have with preprocessing.

## Categories of machine learning

Machine learning can very broadly be divided into four categories: data handling, model design, optimisers, loss selection. In the last session we looked a little bit at model selection, where the capacity/size of the network we were training was an important parameter for reducing overfitting. Model design can also involve choosing different kinds of layers with some kind of inductive bias, but more on this in later sessions. This week we are going to include data handling and loss selection in the exercises.

When training models with machine learning data handling is often more important than the choice of model. 
It is very easy for a model to focus on the "wrong" features while training and end up with a strong bias, or latch onto an easy solution that ignores a lot of information. Even worse, models can fail at their task because it is easier to guess and get the right answer than to learn how to solve the task they are assigned!

## Preprocessing

In the previous exercises you were provided with data which had been "pre-processed" to make it easier for the network to perform well. 

Ideally, all inputs to a neural network are of order 1, centered around a common value.

You can see this as a logical consequence of our dense layers
* Our linear layer $y_j = m_j^{\,\,i} x_i + c_j$ has the weights $m_j^{\,\,i}$ per node $j$ and per input $i$
* Our weights are initialised randomly 
* If our $x_i$ have values which vary drastically, the inputs $x_i$ with the largest values will dominate the outputs $y_j$
* Our gradients will be dominated by a subset of the features
* We won't learn everything we could and can end up with an underperforming model

This issue can be addressed by rescaling the input data such that all of the features are treated equally. 
Any invertible transformation we apply to our training features will leave the optimal solution to the problem unchanged. The solution in the transformed coordinates can always be converted to a solution in the original scale by applying the inverse transformation.
Rescaling the data such that the input features are more suitable for use with a machine learning algorithm is referred to as preprocessing.
Depending on the task and your input data type there are a few standard ways to preprocess the data.

For images:
* Our inputs are pixels with values from 0-255
* We can divide by 255 and ensure our values are between 0 and 1.
* If we are dealing with very dark or very light images, we may add an additional scaling

For distributions:
* We could scale it to be minimum value = 0, maximum value = 1
* We scale using the mean and standard deviation of each feature (dimension)
  * Subtract the mean from all data per feature, and divide by its standard deviation
* We can apply some non-linear scaling (e.g. log) to change the areas of focus
  * It really depends on your physics problem! Does the behaviour change linearly, logarithmically, do you have long tails, spikes?

There is no set in stone best way to preprocess data for all problems. Smart preprocessing can bring you more performance improvements than a brute force bigger network!

### Example

Lets load data similar to what you used last week

In [ ]:
from utils import load_data
import numpy as np
import pandas as pd

df = load_data('data/btag_b_c.h5')
df.head()

Just looking at the preview of the dataset we can see there are a lot of different values and ranges across rows and columns!

Let's now look at a few of the individual distributions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df.columns

In [ ]:
fig, _ = plt.subplots(3, 3, figsize=(14,14))
axes = fig.axes
variables = ['pt','eta','SV1_L3d',
            'IP3D_pu', 'IP3D_pc','IP3D_pb',
            'SV1_masssvx', 'SV1_efracsvx', 'SV1_significance3d']

class_0 = df.loc[df['label'] == 0] # These are our c-jets
class_1 = df.loc[df['label'] == 1] # These are our b-jets
for i, ax in enumerate(axes):
    ax.hist(class_0[variables[i]], bins=20, label='c-jets', histtype='step')
    ax.hist(class_1[variables[i]], bins=20, label='b-jets', histtype='step')
    ax.legend()
    ax.set_xlabel(variables[i])

As you can see, the ranges are widely different, and some of these distributions suffer from having very long tails.

`IP3D_pu/c/b` for example have most values at 0 but with tails up to 1.0.

`pt` has a lot of events with low values but a tail that goes as high as $10^6$!

Let's take a look at their mean and standard deviation.

As we have loaded our data with pandas dataframes we can take advantage of a lot of nice features from there to do this!

In [ ]:
df[variables].mean()

In [ ]:
df[variables].std()

In [ ]:
df.groupby('label')[variables].mean()

In [ ]:
df.groupby('label')[variables].std()

You may also have noticed that the number of events in each class is very different! This will also be a huge problem when training a network.

In [ ]:
frac = df['label'].sum() / df.shape[0]
print(f'{frac * 100:.2f}% of our events are of class 1.')

If you remember, when taking a wild guess with binary cross entropy we expect the loss to be ln(2).
Let's see what happens if we were to guess 0.5 for all of our events in this dataframe.

In [ ]:
def binary_crossentropy(true,pred):
    return -(true*np.log(pred+1e-16)+(1-true)*np.log((1-pred)+1e-16)).mean()

In [ ]:
guess = 0.5*np.ones_like(df.label.values,dtype=np.float32)
print(binary_crossentropy(df.label.values,guess))

It looks like we get our random guess value of ln(2) by randomly guessing on our dataset, and this is us without having learned anything.

But what if we just guess another value, without learning anything from our data?

In [ ]:
plt_range = np.arange(0.05,1.,0.001)
bces = [binary_crossentropy(df.label.values,i) for i in plt_range]
plt.plot(plt_range,bces,label='Fixed guess')
plt.plot(plt_range, 0.69 * np.ones_like(bces),label='Random guess')
plt.xlabel('Fixed guess')
plt.ylabel('Loss')
plt.legend();

Looks like we can minimise our loss just by optimising our fixed guess!

But this isn't machine learning, this is an underlying bias. We don't need anything but the labels to make this prediction! This would then throw away all information in our features!

We can zoom in on this prediction, and we find that the minimum BCE value is well below ln(2).

In [ ]:
plt.plot(plt_range,bces)
plt.xlabel('Fixed guess')
plt.ylim(0,1)
plt.ylabel('Loss')
plt.annotate(f"min: y={np.min(bces):.3f}",(0.8,0.2));

If we aren't careful, our machine learning model will minimise our loss simply by learning the ratio of the class composition.

The more dramatic the imbalance, the harder it will be for a network to see anything else!

For a classification task these are the two main problems we need to address:
* Bias from class imbalance impacting performance
* Tails and large input values impacting performance

Of course, we don't want to throw away any of our data just to balance the ratio of the classes. Instead you can try to apply weights to the binary cross entropy loss, such that the loss values of each class are balanced.

We can build a simple loss by hand that will do this for us.

In [ ]:
def weighted_binary_crossentropy(true,pred,weight0,weight1):
    # Here we apply a weight to each term in the loss function, to change it's relative importance
    return -(true*np.log(pred+1e-16)*weight1 + (1-true)*np.log((1-pred)+1e-16)*weight0).mean()

Now you should derive two weights, one for each class, that will result in the minimum of the blue curve falling at 0.5.

Think about the bias we see above, what it is related to, and how you can derive a simple weight using that information.

Hints:

* You can get the number of events in a dataframe with ```len(df)``` or ```df.shape[0]```
* You can get the number of events in each class by looking at the labels
* You want your weights to sum to 1/(number of classes) for the balanced loss here

In [ ]:
#weight_0 = 
#weight_1 = 

In [ ]:
plt_range = np.arange(0.05,1.,0.001)
bces = [weighted_binary_crossentropy(df.label.values,i,weight_0, weight_1) for i in plt_range]
plt.plot(plt_range,bces,label='Fixed guess')
plt.plot(plt_range, 0.69 * np.ones_like(bces),label='Random guess')
plt.xlabel('Fixed guess')
plt.ylabel('Loss')
plt.legend();

Hopefully you can now see that random guessing is again the optimal solution like this one below:

![fixed bias](data/random_solution.png "Fixed bias")

So we have fixed the problem we had with our class imbalance by carefully choosing our loss. Tensorflow offers an easy way to apply these weights during training, and scikit learn will give you a simple function to calculate the weights. But we will leave the implementation of this during the training up to you.

In tensorflow with keras, when using the "fit" function you just need to pass them as an option ```class_weight```.

Equally, instead of applying weights per class, you can calculate them per sample.

Can you think of how to trivially extend your current solution to be a per-sample weight?

In tensorflow with keras you can pass an extra set of values in the fit function with ```sample_weight``` and it needs to have the same length as your inputs and outputs.

Can you think of benefits of having a sample weight instead of just a class weight?

### Exercises

Build a classifier to separate the two classes using the variables defined above as `variables`.

You will want to make sure you are dealing with numpy arrays (with pandas dataframes you can use `.values()`)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.optimizers import Adam

#You will need to fill in the blanks!
#ins = Input(shape=(9,))
#h = Dense(...)(ins)
#h = Dense(...)(h)
#outs = Dense(...)

#opt = Adam(lr=learning_rate)

# model = Model(ins=ins,outs=outs)
# You can also pass class weights here instead of when fitting!
# model.compile(optimizer=opt,loss='bce')

### it is also possible to add metrics when compiling to keep track of the models performance while training
### just add an option `metrics` with a list of what you want, i.e. metrics=['acc','AUC']

When we train a classifier we will always use a training and validation set (with the latter used to assess its performance). Ideally we will also have a test set which we use after fully optimising our classifier, this will be the final measure of performance for when we apply our to the real data we collect.

Remember, never train your classifier with data you will then apply it on. This is also true for the validation set.

In these examples, we will focus only on splitting our data into train and validation, with a 50:50 split.

### Set up the data for training and validation

Load your data and make sure it is shuffled. As it is already loaded, lets just shuffle the dataframe.
* Depending on your data format, you make have to use different functions
* With pandas dataframes we can do this with `df = df.sample(frac=1)`
  * You can use df.head() to return the top 5 rows of the dataframe to check this has shuffled the data!

In [ ]:
# shuffle the dataframe
df = df.sample(frac=1)

Now split the data into two sets, training and validation, with input variables and labels in two separate arrays.

* Use the variables array we defined earlier to select the variables, and use the label column for the labels.


Tip:

You can split arrays/tensors/dataframes in python with the slice operator `:`

So for example
```x_train = x[:len(x)//2]
x_val = x[len(x)//2:]```

Remember, you will need to use an int for indices, so make sure to take care of odd numbers of events

In [ ]:
# x_train = df[variables].values[:len(df)//2]

In [ ]:
# x_train =
# x_val = 
# y_train =
# y_val =
### And if you are using sample weights
### w_train = 
### w_val = 

Now that you have your model compiled and your data loaded, train the classifier with `model.fit`!

You can pass both the training and validation sets to the classifier.

Or if you want to use only the training data, you can pass that with the option `validation_split` to skim off some of the train data for the validation set. This is sometimes useful for quick studies, but it is better to predefine your datasets to ensure reproducibility.

In [ ]:
#history = model.fit(.....)

You can plot the loss and accuracy values from the model you have trained from the history object
`history.history['loss']`,`history.history['val_loss']`,`history.history['acc']`,`history.history['val_acc']`. You can see what is available with

In [ ]:
print(history.history.keys())

It is very useful to plot the loss development to check that you have not overtrained the model!

You can also now perform extra studies on the performance of the model, looking at the ROC curves or efficiencies as a function of other quantities.

First get all your predictions for the validation set and then plot the ROC curves using
`sklearn.metrics.roc_curve`

Tip: you can always check in the sklearn documentation how to use a function, but if you want to do it inline you can use: `help(function_name)`

In [ ]:
pred_val = model.predict(x_val)

In [ ]:
from sklearn.metrics import roc_curve

fpr,tpr,thresh = roc_curve(.....)

In [ ]:
plt.plot(tpr,fpr,label='mylabel')
plt.xlabel('Signal Efficiency')
plt.ylabel('Background Efficiency')
plt.legend()

If you like, you can compare this to a training where you don't use the class weights!

Plot both ROC curves on the same plot with different labels to compare them.

In [ ]:
#mymodel2 = ....
#mymodel2.compile(...)
#mymodel2.fit(...)
#pred_val_noweight = mymodel2.predict(x_val)

In [ ]:
#fpr_nw,tpr_nw,thresh_nw = roc_curve(....)
# plt.plot(tpr,fpr,label='weighted')
# plt.plot(tpr_nw,fpr_nw,label='no weights')
# plt.xlabel('Signal Efficiency')
# plt.ylabel('Background Efficiency')
# plt.legend()

## Preprocessing input ranges

As mentioned earlier, networks prefer it when inputs are scaled and it can help them converge faster or lead to better performance.

Now that you have a balanced classifier, let's look at the input variables again.

We know how to calculate the mean and standard deviation, so lets apply this to the variables we are interested in.
We should calculate this on the training set and apply it to the validation set.

In [ ]:
mu_scale = np.mean(x_train,axis=0,keepdims=True)#axis specifies which column to perform over
std_scale = np.std(x_train,axis=0,keepdims=True)#keepdims keeps the same number of axes after the operation (remember broadcasting!)
x_train_t = (x_train - mu_scale)/std_scale
x_val_t = (x_val - mu_scale)/std_scale

Now you can train a classifier again, but now with these new values for the inputs, and compare it to the previous performance.
If you ever want to reset a network to an untrained state in tf.keras, recompile it.

It is not guaranteed which preprocessing will be best out of the box when training a network, and comparing many is always worthwhile!

[Scikit learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) has a great number of preprocessing functions available, feel free to try several of them and compare!

In [ ]:
#define classifier

In [ ]:
#train classifier

In [ ]:
#evaluate performance

In [ ]:
#compare performance

## Balancing against input variables

Sometimes when we train a classifier we want it to separate classes based on some observables but not based on others.

In flavour tagging this is the case with the momentum and angle of the jet - these are our first to variables in the network, $p_T$ and $\eta$!

Unfortunately, if we look at these two features they have different distributions, big problem!

If we plot the correlation between the output and one of these two variables we will see there is a trend.

To combat this, we can treat it just like we did the class bias and derive a weight per sample as a function of these distributions.

To make distributions match we can choose one as a reference, and then work out weights to multiple the other one by to get it to match.

Simple method:
* Let's use histograms!
* Make a histogram for each class (nominal and default) with fine binning (as fine as statistics allows us)
* Take a ratio of the number of events in each bin between the nominal and target distributions
* Use the ratio as our per-bin weight
* Assign each event in the nominal class based on its value

With this approach in one dimension we can get pretty close, in two dimensions we start getting limited by statistics, and beyond that we need to be much smarter.

We can also use downsampling or other pdf based methods

In [ ]:
#create the plot and get the values per bin
#we are using weights to normalise the histograms to sum to unity in case we want to use the class weights in training
x_train_sig = x_train[y_train==1]
x_train_bkg = x_train[y_train==0]
bkg_vals,ptbins,_= plt.hist(x_train_bkg[:,0], weights=np.ones(len(x_train_bkg))/len(x_train_bkg), bins=np.arange(0,500e3,10e3),histtype='step',linewidth=2, label='c-jets')
signal_vals,ptbins,_=plt.hist(x_train_sig[:,0], weights=np.ones(len(x_train_sig))/len(x_train_sig), bins=np.arange(0,500e3,10e3),histtype='step',linewidth=2, label='b-jets')
plt.xlabel('$p_T$ [MeV]')
plt.ylabel('Normalised Entries')
_=plt.legend()

In [ ]:
#signal_vals = weights * bkg_vals, ergo
binweights = signal_vals / bkg_vals

Oh dear, we get divide by zero! We will need to be careful about our binning.

These are all coming from the very long tail we have in the histogram, and the empty bins at the beginning of the histogram.

We can deal with this by lumping them all into an overflow bin and calculating a common weigh.

Or, because we know it is just a very small effect in the tail, we can set the nan values to 1.0

In [ ]:
binweights = np.nan_to_num(binweights,nan=1.0,posinf=1.0,neginf=-1)

Now we can calculate a weight per event by looking up which bin they are in with a simple loop (there are better ways!)

In [ ]:
w_train = np.ones(len(x_train))
for i in range(len(binweights)-1):
    w_train[(x_train[:,0] >= ptbins[i]) & (x_train[:,0] < ptbins[i+1]) & (y_train == 0)] *= binweights[i]

In [ ]:
#cross check that our weights make sense!
print(np.unique(w_train[y_train==1]))
print(np.unique(w_train[y_train==0]))

In [ ]:
#create the plot and get the values per bin
#we are using weights to normalise the histograms to sum to unity in case we want to use the class weights in training
x_train_sig = x_train[y_train==1]
x_train_bkg = x_train[y_train==0]
bkg_vals,ptbins,_= plt.hist(x_train_bkg[:,0], weights=w_train[y_train==0]*np.ones(len(x_train_bkg))/len(x_train_bkg), bins=np.arange(0,500e3,10e3),histtype='step',linestyle='-',linewidth=2,label='c-jets')
signal_vals,ptbins,_=plt.hist(x_train_sig[:,0], weights=w_train[y_train==1]*np.ones(len(x_train_sig))/len(x_train_sig), bins=np.arange(0,500e3,10e3),histtype='step',linestyle='--',linewidth=2,label='b-jets')
plt.xlabel('$p_T$ [MeV]')
plt.ylabel('Normalised Entries')
_=plt.legend()

Now we can see that the distributions perfectly match for this binning!

If you change the binning you will see some small discrepancies appear but this is much better than before.

Now you can retrain the classifier and see what happens to the performance - do you expect it to drop or increase?

You should now also look at the ROC curves in bins of $p_T$ (use the slice operator to separate your validation set based on this and recalculate the ROC curve each time.

You can now also do the same for $\eta$, and in principle both simultaneously!

In [ ]:
# Train classifier

In [ ]:
# Evaluate classifier

In [ ]:
# Compare performance

In [ ]:
# Hint
# Look at ROC curves in different bins of pT with
# bins = np.arange(25e3,150e3,25e3)
# for lower,upper in zip(bins[:-1],bins[1:]):
#     x_val_t[(x_val[:,0] >= lower) & (x_val[:,0] < upper)]

### Remark

One other problem with datasets is sometimes you have variables that aren't always defined.

The data you have used so far has had these removed but there are several ways to handle them:

* Don't use variables which can have nan values
* Remove any event with nan values
  * Assign all events with nan values to a class or pretend they don't exist
* Set all nan values to a default value
  * Use the mean value
  * Motivate it from the physics you are looking at
  * You may want to indicate to the network that you have done this with an extra variable which is 1 or 0 depending on whether you did this